In [13]:
import pandas as pd
import numpy as np
from sklearn import preprocessing, decomposition, metrics, model_selection
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE
from math import radians, cos, sin, asin, sqrt
import pickle
from utils import *
from bayes_opt import BayesianOptimization

from numpy import asarray
from numpy import savetxt

In [3]:
import xgboost as xgb

In [4]:
def preprocess1(data, is_train=True, use_synthetic_generation=True):
    #Get columns list and remove tripid col and non numerical cols
    cols = list(data.columns)
    cols.remove("tripid")
    cols.remove("pickup_time")
    cols.remove("drop_time")    
    data = data.drop(columns=["tripid"], axis=1)
    
    if is_train:
        #Adjust target label to True or False
        data.loc[data.label == "correct", "label"] = True
        data.loc[data.label == "incorrect", "label"] = False
        data["label"] = data["label"].astype('bool')
        cols.remove("label")
        
    #Fill missing values
    if is_train:
        dataset = fill_missing_val(data, cols)
    else:
        dataset = data
    
    #calculate the distance
    dataset = calculate_distance(dataset)
    
    #Create hour value for timestamps
    dataset["pickup_time"] = data.apply((lambda x: get_hour(x["pickup_time"], "pickup_time")), axis=1).astype("int")
    dataset["drop_time"] = data.apply((lambda x: get_hour(x["drop_time"], "pickup_time")), axis=1).astype("int")

    
    if is_train and use_synthetic_generation:
        #Use SMOTE to resample
        datasetX, datasetY = balance_using_SMOTE(dataset.drop(columns=["label"], axis=1), dataset["label"])
    elif is_train:
        datasetX, datasetY = dataset.drop(columns=["label"], axis=1), dataset["label"]
    else:
        datasetX = dataset
    
    #Bin timestamps
    datasetX = get_binned_DF(datasetX, "pickup_time")
    datasetX = get_binned_DF(datasetX, "drop_time")

    
    #Scale dataset
    datasetX = scale_data(datasetX)
    
    if is_train:
        return datasetX, datasetY
    
    return datasetX

In [5]:
df = pd.read_csv("train.csv")
X, Y = preprocess1(df, use_synthetic_generation=False)

In [9]:
x, x_validate, y, y_validate = model_selection.train_test_split(X, Y, test_size=0.1, random_state=42)
x_tr, x_te, y_tr, y_te = model_selection.train_test_split(x, y, test_size=0.175, random_state=42)

In [10]:
eval_set = [(x_tr, y_tr), (x_te, y_te)]
eval_metric = ["auc","error"]

In [19]:
tree_no = 0
best = 0
model_type = 'XGBoost'

In [20]:
train = xgb.DMatrix(x_tr, label=y_tr)
test = xgb.DMatrix(x_te, label=y_te)

In [27]:
def tree_optimization(eta, gamma, max_depth, subsample, lambda_val, num_parallel_tree):
    global train, test, tree_no, model_type, best

    booster_params = {
        'eta': eta,
        'gamma': gamma,
        "eval_metric": "auc",
        'max_depth': int(np.around(max_depth)),
        'subsample': subsample,
        'sampling_method': 'gradient_based',
        'alpha': lambda_val,
        'grow_policy': 'lossguide',
        'num_parallel_tree': int(num_parallel_tree),
        'objective': 'binary:logistic',
        'verbosity': 1,
    }

    results = {}

    print("generating model")
    model = xgb.train(booster_params, train, num_boost_round=100, evals=[(test, 'val')],
                               early_stopping_rounds=10, evals_result=results)
    num_iter = model.best_iteration + 1
    best_score = model.best_score

    if best_score > best:
        print(f"Score incresed to {best_score} from {best}")
        best = best_score
        pickle.dump(model, open(f"xgb_model_{tree_no}", 'wb'))
        model = xgb.train(booster_params, train, num_boost_round=num_iter, evals=[(test, 'val')],
                               evals_result=results)
        model.save_model(f'logs/H1N1_{tree_no}_AUC_{best_H1N1}.model')

    score = best_score
    
    tree_no += 1
    return score

In [28]:
def use_bayesian_optimization():
    global model_type
    max_file = open(f"max_params_{model_type}.txt", 'w')

    NN_BAYESIAN = BayesianOptimization(
        tree_optimization,
        {
            'eta': (0, 1),
            'gamma': (0, 2),
            'max_depth': (5, 15),
            'subsample': (0, 1),
            'lambda_val': (0, 3),
            'num_parallel_tree': (7, 17)
        })
    NN_BAYESIAN.maximize(init_points=50, n_iter=550, acq='ei', xi=0.0)
    write_to_log(NN_BAYESIAN.max)
    print('Best NN parameters: ', NN_BAYESIAN.max, file=max_file)

In [29]:
use_bayesian_optimization()

|   iter    |  target   |    eta    |   gamma   | lambda... | max_depth | num_pa... | subsample |
-------------------------------------------------------------------------------------------------
generating model
[02:38:02] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1588600955503/work/src/learner.cc:328: 
Parameters: { sampling_method } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	val-auc:0.88186
Will train until val-auc hasn't improved in 10 rounds.
[1]	val-auc:0.88298
[2]	val-auc:0.88369
[3]	val-auc:0.88397
[4]	val-auc:0.88393
[5]	val-auc:0.88393
[6]	val-auc:0.88387
[7]	val-auc:0.88388
[8]	val-auc:0.88454
[9]	val-auc:0.88466
[10]	val-auc:0.88615
[11]	val-auc:0.88619
[12]	val-auc:0.88610
[13]	val-auc:0.88614
[14]	val-auc:0.88613
[15]	val-auc:0.88613
[16]

[0]	val-auc:0.88462
Will train until val-auc hasn't improved in 10 rounds.
[1]	val-auc:0.89306
[2]	val-auc:0.90328
[3]	val-auc:0.92447
[4]	val-auc:0.93818
[5]	val-auc:0.94236
[6]	val-auc:0.94272
[7]	val-auc:0.94183
[8]	val-auc:0.94096
[9]	val-auc:0.94146
[10]	val-auc:0.94107
[11]	val-auc:0.94038
[12]	val-auc:0.93998
[13]	val-auc:0.93951
[14]	val-auc:0.93928
[15]	val-auc:0.93906
[16]	val-auc:0.93908
Stopping. Best iteration:
[6]	val-auc:0.94272

|  4        |  0.9427   |  0.7476   |  0.1549   |  1.634    |  12.6     |  10.38    |  0.6944   |
generating model
[02:39:11] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1588600955503/work/src/learner.cc:328: 
Parameters: { sampling_method } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	val-auc:0.86535
Will train unt

[32]	val-auc:0.93451
[33]	val-auc:0.93416
[34]	val-auc:0.93434
[35]	val-auc:0.93497
[36]	val-auc:0.93437
[37]	val-auc:0.93446
[38]	val-auc:0.93422
[39]	val-auc:0.93321
[40]	val-auc:0.93269
[41]	val-auc:0.93275
[42]	val-auc:0.93265
[43]	val-auc:0.93329
[44]	val-auc:0.93287
[45]	val-auc:0.93260
Stopping. Best iteration:
[35]	val-auc:0.93497

|  9        |  0.935    |  0.483    |  1.54     |  2.574    |  7.152    |  16.5     |  0.1028   |
generating model
[02:40:13] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1588600955503/work/src/learner.cc:328: 
Parameters: { sampling_method } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	val-auc:0.88486
Will train until val-auc hasn't improved in 10 rounds.
[1]	val-auc:0.89834
[2]	val-auc:0.90996
[3]	val-auc:0.92499
[4]	va

[0]	val-auc:0.88259
Will train until val-auc hasn't improved in 10 rounds.
[1]	val-auc:0.90398
[2]	val-auc:0.90556
[3]	val-auc:0.91911
[4]	val-auc:0.93187
[5]	val-auc:0.93603
[6]	val-auc:0.93846
[7]	val-auc:0.93773
[8]	val-auc:0.93745
[9]	val-auc:0.93988
[10]	val-auc:0.94013
[11]	val-auc:0.93635
[12]	val-auc:0.93767
[13]	val-auc:0.93804
[14]	val-auc:0.93847
[15]	val-auc:0.93870
[16]	val-auc:0.93830
[17]	val-auc:0.93753
[18]	val-auc:0.93791
[19]	val-auc:0.93674
[20]	val-auc:0.93709
Stopping. Best iteration:
[10]	val-auc:0.94013

|  15       |  0.9401   |  0.6781   |  1.069    |  0.11     |  12.42    |  10.05    |  0.1432   |
generating model
[02:41:14] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1588600955503/work/src/learner.cc:328: 
Parameters: { sampling_method } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verificatio

Will train until val-auc hasn't improved in 10 rounds.
[1]	val-auc:0.88869
[2]	val-auc:0.88973
[3]	val-auc:0.91667
[4]	val-auc:0.92605
[5]	val-auc:0.93132
[6]	val-auc:0.93306
[7]	val-auc:0.93439
[8]	val-auc:0.93671
[9]	val-auc:0.93873
[10]	val-auc:0.93890
[11]	val-auc:0.93782
[12]	val-auc:0.93756
[13]	val-auc:0.93769
[14]	val-auc:0.93850
[15]	val-auc:0.93889
[16]	val-auc:0.94046
[17]	val-auc:0.93949
[18]	val-auc:0.93958
[19]	val-auc:0.93822
[20]	val-auc:0.93944
[21]	val-auc:0.93893
[22]	val-auc:0.93865
[23]	val-auc:0.93803
[24]	val-auc:0.93930
[25]	val-auc:0.93913
[26]	val-auc:0.93897
Stopping. Best iteration:
[16]	val-auc:0.94046

|  21       |  0.9405   |  0.8184   |  0.4423   |  2.524    |  6.304    |  10.4     |  0.1748   |
generating model
[02:42:12] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1588600955503/work/src/learner.cc:328: 
Parameters: { sampling_method } might not be used.

  This may not be accurate due to some parameters are only used in language bindin

...

Will train until val-auc hasn't improved in 10 rounds.
[1]	val-auc:0.50000
[2]	val-auc:0.50000
[3]	val-auc:0.50000
[4]	val-auc:0.50000
[5]	val-auc:0.50000
[6]	val-auc:0.50000
[7]	val-auc:0.50000
[8]	val-auc:0.50000
[9]	val-auc:0.50000
[10]	val-auc:0.50000
Stopping. Best iteration:
[0]	val-auc:0.50000

|  575      |  0.5      |  0.0      |  2.0      |  0.0      |  12.0     |  7.0      |  0.8178   |
generating model
[04:00:55] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1588600955503/work/src/learner.cc:328: 
Parameters: { sampling_method } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	val-auc:0.87162
Will train until val-auc hasn't improved in 10 rounds.
[1]	val-auc:0.88520
[2]	val-auc:0.91692
[3]	val-auc:0.93219
[4]	val-auc:0.93354
[5]	val-auc:0.93011
[6]	v

Will train until val-auc hasn't improved in 10 rounds.
[1]	val-auc:0.89741
[2]	val-auc:0.91303
[3]	val-auc:0.92628
[4]	val-auc:0.93158
[5]	val-auc:0.93663
[6]	val-auc:0.94256
[7]	val-auc:0.94383
[8]	val-auc:0.94305
[9]	val-auc:0.94110
[10]	val-auc:0.94101
[11]	val-auc:0.94054
[12]	val-auc:0.94024
[13]	val-auc:0.93931
[14]	val-auc:0.93956
[15]	val-auc:0.94009
[16]	val-auc:0.93988
[17]	val-auc:0.93969
Stopping. Best iteration:
[7]	val-auc:0.94383

|  583      |  0.9438   |  0.4955   |  2.0      |  0.0      |  14.44    |  8.066    |  0.6425   |
generating model
[04:01:54] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1588600955503/work/src/learner.cc:328: 
Parameters: { sampling_method } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	val-auc:0.88219
Will train un

[7]	val-auc:0.91592
[8]	val-auc:0.92113
[9]	val-auc:0.93117
[10]	val-auc:0.93440
[11]	val-auc:0.93844
[12]	val-auc:0.93963
[13]	val-auc:0.94019
[14]	val-auc:0.94098
[15]	val-auc:0.94170
[16]	val-auc:0.94205
[17]	val-auc:0.94185
[18]	val-auc:0.94139
[19]	val-auc:0.94162
[20]	val-auc:0.94219
[21]	val-auc:0.94283
[22]	val-auc:0.94305
[23]	val-auc:0.94277
[24]	val-auc:0.94280
[25]	val-auc:0.94274
[26]	val-auc:0.94261
[27]	val-auc:0.94275
[28]	val-auc:0.94241
[29]	val-auc:0.94215
[30]	val-auc:0.94224
[31]	val-auc:0.94202
[32]	val-auc:0.94207
Stopping. Best iteration:
[22]	val-auc:0.94305

|  590      |  0.9431   |  0.3364   |  1.261    |  2.56     |  14.27    |  11.15    |  0.9935   |
generating model
[04:03:10] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1588600955503/work/src/learner.cc:328: 
Parameters: { sampling_method } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some pa

Will train until val-auc hasn't improved in 10 rounds.
[1]	val-auc:0.50000
[2]	val-auc:0.50000
[3]	val-auc:0.50000
[4]	val-auc:0.50000
[5]	val-auc:0.50000
[6]	val-auc:0.50000
[7]	val-auc:0.50000
[8]	val-auc:0.50000
[9]	val-auc:0.50000
[10]	val-auc:0.50000
Stopping. Best iteration:
[0]	val-auc:0.50000

|  598      |  0.5      |  0.0      |  2.0      |  3.0      |  8.595    |  7.0      |  1.0      |
generating model
[04:04:27] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1588600955503/work/src/learner.cc:328: 
Parameters: { sampling_method } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	val-auc:0.88684
Will train until val-auc hasn't improved in 10 rounds.
[1]	val-auc:0.89523
[2]	val-auc:0.92250
[3]	val-auc:0.93354
[4]	val-auc:0.93867
[5]	val-auc:0.94204
[6]	v

NameError: name 'write_to_log' is not defined

In [131]:
evalutate_f1(model2, x_te, y_te)

0.9744144144144145

In [67]:
models, scores = train_k_fold_xgb(X, Y, k=10, use_synthetic_generation=False)

0.991025641025641 0.9763332281476805
0.9916325440626669 0.9730927508705285
0.9923522925337034 0.971282705807275
0.9914195179264429 0.9743751977222398
0.9921018926995873 0.9712092130518234
0.9917784816884365 0.9742284441616291
0.9916720051249199 0.9739351557533377
0.9911051021134277 0.9794238683127572
0.9922690512665217 0.9747793190416141
0.9918635636880442 0.9691516709511567


In [50]:
models, scores = train_k_fold_xgb(X, Y, k=10, use_synthetic_generation=False)

0.991025641025641 0.9763332281476805
0.9916325440626669 0.9730927508705285
0.9923522925337034 0.971282705807275
0.9914195179264429 0.9743751977222398
0.9921018926995873 0.9712092130518234
0.9917784816884365 0.9742284441616291
0.9916720051249199 0.9739351557533377
0.9911051021134277 0.9794238683127572
0.9922690512665217 0.9747793190416141
0.9918635636880442 0.9691516709511567


In [15]:
temp_model = models[2]

In [51]:
temp_model2 = models[7]

In [44]:
modelCount = 0
for model in models:
    count = 0
    print("MODEL:", modelCount)
    for imp in temp_model2.feature_importances_:
        print(cols2[count], imp)
        count += 1
    modelCount += 1

MODEL: 0
0 0.10220718
1 0.082426496
2 0.02722826
3 0.029743383
4 0.019243227
5 0.054467015
6 0.04218942
7 0.011823909
8 0.004366155
9 0.0
10 0.008116734
11 0.003055509
12 0.008724934
13 0.02266054
14 0.008568795
15 0.01259461
16 0.022347387
17 0.016784746
18 0.014412417
19 0.020254219
20 0.016610643
21 0.025419962
22 0.010967209
23 0.013484519
24 0.0172702
25 0.023546627
26 0.013625785
27 0.0067540393
28 0.024642622
29 0.012295193
30 0.0096653765
31 0.013180087
32 0.00732311
33 0.007363757
34 0.0
35 0.0055287043
36 0.0
37 0.010399229
38 0.014921948
39 0.019594578
40 0.030434474
41 0.026652468
42 0.015903128
43 0.026930898
44 0.018275006
45 0.010642718
46 0.02797758
47 0.01425777
48 0.017561518
49 0.018236607
50 0.015835816
51 0.007858519
52 0.0056250333
53 0.0
54 0.0
MODEL: 1
0 0.10220718
1 0.082426496
2 0.02722826
3 0.029743383
4 0.019243227
5 0.054467015
6 0.04218942
7 0.011823909
8 0.004366155
9 0.0
10 0.008116734
11 0.003055509
12 0.008724934
13 0.02266054
14 0.008568795
15 0.01259

In [115]:
pickle.dump(model, open("xgb_model1", 'wb'))

In [30]:
loaded_model = pickle.load(open("xgb_model_551", 'rb'))

In [34]:
loaded_model.predict(validate)

array([0.910475  , 0.96718204, 0.90931684, ..., 0.98657787, 0.99764866,
       0.9956019 ], dtype=float32)

In [32]:
validate = xgb.DMatrix(x_validate, label=y_validate)

In [40]:
np.mean(y_validate == (loaded_model.predict(validate) > 0.5))

0.9534342258440046

In [41]:
test_data = pd.read_csv("test.csv")
X_pred = preprocess1(test_data, False, False)
X_pred.columns

RangeIndex(start=0, stop=55, step=1)

In [117]:
predictions = model.predict(X_pred)

In [52]:
test_results = pd.DataFrame(loaded_model.predict(xgb.DMatrix(X_pred)) >= 0.5, columns=["prediction"])

In [53]:
test_results["tripid"] = test_data["tripid"]
test_results["prediction"] = test_results["prediction"].astype("int")
test_results.head()

,prediction,tripid
0,1,213284604
1,1,213286352
2,1,213293973
3,1,213294622
4,1,213298687


In [54]:
test_results.groupby("prediction").count()

,tripid
prediction,
0,583
1,7993


In [55]:
test_results[["tripid", "prediction"]].to_csv("predictions7.csv", index=False)